# transfert des poids

Dans ce tutoriel, vous apprendrez à classer des images de chats et de chiens en utilisant l'apprentissage par transfert (=transfert learning). Il s'agit de réutiliser un modèle pré-entrainé.



## Settings

In [1]:
%reset -f

In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds
import scipy

In [3]:
def show_15_img(images):

    assert len(images)>=15

    fig,axs=plt.subplots(3,5,figsize=(10,6))
    axs=axs.flatten()

    for ax,img in zip(axs,images[:15]):
        ax.imshow(img)
        ax.axis('off')

    fig.tight_layout()

## Théorie




### Principe

On va utilisé un modèle déjà entrainé  sur un grand ensemble de données. Les caractéristiques spatiales qu'il a apprise  agiront efficacement comme un modèle générique de notre monde visuel.


Nous allons utiliser le modèle **MobileNet V2** entrainé sur ImageNet. On peut reconditionner ce réseau pour qu'il classe des choses qui ne sont pas présente sur imagenet. Par exemple des meubles, ou même des cellules.

Ce "transfert-learning" rend le deep-learning efficace même pour sur des jeux de données de petite taille.


Mais revenons à nous moutons, ou plutôt à nos chats et cheins.  Imagenet contient des tas d'espèces animales qui ont des pellages, des oreilles, des queues... MobilNet a appris à transformer toute ces caractérisiques d'animaux en des `features` qui lui sont propre:  ce sont de très nombreuses petites images abstraites qui se dessinent quand Mobilnet voit une image.  En recombinant ces features, un classifier poura différencier les  chats et les chiens avec très peu d'erreur.











### Garder les couches convolutives




Un convnet comportent deux parties :
* 1/ une série de convolutions
* 2/ un classifier fait de couche denses.

On ne va garder que les couches convolutives en haut desquelles on va greffer un nouveau classifier.

![échange de classificateurs FC](https://s3.amazonaws.com/book.keras.io/img/ch5/swapping_fc_classifier.png)


***A vous:*** Pourquoi ne garde-t-on pas le classifier initial ?











Les couches convolutives n'ont pas toute le même rôle:
* Les couches qui viennent plus tôt dans le modèle extraient des cartes d'entités locales très génériques (comme les bords, les couleurs et les textures)
* les couches supérieures extraient des concepts plus abstraits (tels que "l'oreille de chat" ou "l'œil de chien").


Ainsi, si vos données diffère beaucoup des données initiales il est préférable de supprimer aussi les couches convolutives du haut.

## Les données

### Téléchargement

Utilisons [TensorFlow Datasets](http://tensorflow.org/datasets) pour charger le dataset "cats and dogs". Il s'agit ici du dataset complet de kaggle (pas d'une réduction).




La méthode `tfds.load` télécharge et met en cache les données.

Puisque `"cats_vs_dogs"` ne définit pas de splits standards, utilisons la fonction de subsplit pour le diviser en (train, validation, test) avec respectivement 80%, 10% et 10% des données.


In [4]:
(raw_train, raw_validation, raw_test), metadata = tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
    with_info=True,
    as_supervised=True,
)

In [5]:
print(raw_train)
print(raw_validation)
print(raw_test)

Nous avons créer 3 objets de type `tf.data.Dataset`, chacun contenant des paires `(image, label)` où les images ont des tailles variables et les label sont des 0 ou 1 indiquant la catégorie chat ou chien.


Voici les  premiers élément de ce dataset

In [6]:
get_label_name = metadata.features['label'].int2str

for image, label in raw_train.take(2):
  plt.figure()
  plt.imshow(image)
  plt.title(get_label_name(label))

### Formatage

Utilisons le module `tf.image` pour formater les images:

* Redimensionnons les images à une taille d'entrée fixe
* redimensionnones les canaux d'entrée dans une plage de `[-1,1]`.


In [8]:
IMG_SIZE = 160 # All images will be resized to 160x160

def format_example(image, label):
  image = tf.cast(image, tf.float32)
  """
  normalisation inhabituelle. Les pixels sont envoyés dans [-1,1]
  J'imagine que c'est cette normalisation qui a été employée durant le pré-entrainement
  """
  image = (image/127.5) - 1
  image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
  return image, label

On précise le prétraitement à faire sur chaque image.

In [9]:
train = raw_train.map(format_example)
validation = raw_validation.map(format_example)
test = raw_test.map(format_example)

In [10]:
get_label_name = metadata.features['label'].int2str

for image, label in train.take(2):
  plt.figure()
  plt.imshow(image)
  plt.title(get_label_name(label))

Mélangeons et séparons les données en batch.

In [11]:
BATCH_SIZE = 32
SHUFFLE_BUFFER_SIZE = 1000

In [12]:
train_batches = train.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
validation_batches = validation.batch(BATCH_SIZE)
test_batches = test.batch(BATCH_SIZE)

Observons un batch:

In [13]:
for image_batch, label_batch in train_batches.take(1):
   pass

image_batch.shape

## Observation des features-maps

### Récupération de MobileNet

In [14]:
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)

# Create the base model from the pre-trained model MobileNet V2
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

In [15]:
base_model.summary()

[explication sur mobilNet](https://machinethink.net/blog/mobilenet-v2/)

Ce modèle convertit chaque image `160x160x3` en un bloc de features `5x5x1280`.

In [16]:
feature_batch = base_model(image_batch)
print(feature_batch.shape)

### Effet d'un décalage de l'image initiale

Les features-maps produitent par des convolutions gardent en partie localisation spaciales des features d'une image. Et plus la feature-map est proche de l'entrée, et plus cette localisation est précise (les maxpooling/stride réduisent la résolution).   

Pour les problèmes où la localisation de l'objet est importante il est préférable d'utiliser des couches assez basses ou alors de créer des racoursis. C'est sur ce principe qu'est basé le Unet (conçu pour de la segmentation d'image, où la localisation est primordiale).

In [17]:
img0=(image_batch[0])
img0.shape
imgs=np.empty(shape=(15,)+img0.shape)
decay=np.linspace(-50,50,15)
for i,param in enumerate(decay):
    imgs[i,:,:,:]=scipy.ndimage.shift(img0,shift=[param,param,0],order=1)

In [18]:
show_15_img((imgs+1)/2)

In [19]:
base_model.get_layer("block_1_depthwise_relu").output

In [20]:
intermediate_layer_model = tf.keras.Model(inputs=base_model.input,
                                 outputs=base_model.get_layer("block_1_depthwise_relu").output)

In [21]:
intermediate_output = intermediate_layer_model.predict(imgs)
intermediate_output.shape

In [22]:
show_15_img([small[:,:,10] for small in intermediate_output])

In [24]:
final_features = base_model(imgs)
show_15_img([small[:,:,100] for small in final_features])
#ou bien
#show_15_img(final_features[:15,:,:,10])

***A vous:*** Observer les features d'une image tournée. Faites en sorte que les rotations de 45, 90, et 180 degrés apparaissent.

## Classification



### Geler les couches convolutive

Nous gelons la base convolutive qui servira  d'extracteur de caractéristiques. On passera les images dans ce réseau comme si on faisait un pré-traitement classique (ex: changement d'espace des couleurs, normalisation, rescaling).

In [25]:
base_model.trainable = False

In [26]:
base_model.trainable_variables

### Ajoutons une tête classifieuse

Pour générer des prédictions à partir d'un bloc de feature de shape (5,5,1280), nous faisons la moyenne de toute les imagettes `5x5` ce qui donne un simple vecteur de taille 1280.


 On évite d'applatir pour éviter que la couche dense d'après soit immense.


In [27]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
#ou en tf:
#feature_batch_average=tf.reduce_???(feature_batch,axis=[???])
print(feature_batch_average.shape) #(b,1280)

***A vous:*** Remplacer les ??? pour faire du  `GlobalAveragePooling2D` avec du pure tensorflow.

Appliquons une couche Dense pour convertir ce vecteur une seule prédiction par image.

In [28]:
prediction_layer = tf.keras.layers.Dense(1)
prediction_batch = (feature_batch_average)
print(prediction_batch.shape)

Nous empilons le tout:

In [29]:
model = tf.keras.Sequential([
  base_model,
  global_average_layer,
  prediction_layer
])

In [30]:
def my_loss(y_true,y_pred):
    return tf.keras.losses.binary_crossentropy(y_true,y_pred,from_logits=True)

In [31]:
def my_accuracy(y_true,y_pred):
    return tf.keras.metrics.binary_accuracy(y_true,y_pred,threshold=0)

In [35]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.optimizers.RMSprop(base_learning_rate),
              loss=my_loss,
              metrics=[my_accuracy])

***A vous:***
* Pourquoi est-ce qu'on n'a pas mis de fonction d'activation dans la dernière couche.  
* Que signifie `threshold=0.0` ?
* Y a-t-il un threshold dans les categorical_accuracy ?

### Entrainons le model

On arrive très vite à une très bonne accuracy.


In [37]:
initial_epochs=5

In [38]:
history = model.fit(train_batches,
                    epochs=initial_epochs,
                    validation_data=validation_batches)

### Courbes d'apprentissage


In [39]:
history.history

In [46]:
acc = history.history['my_accuracy']
val_acc = history.history['val_my_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']


fig,(ax0,ax1)=plt.subplots(2,1,figsize=(8,8))

ax0.plot(acc, label='Training Accuracy')
ax0.plot(val_acc, label='Validation Accuracy')
ax0.legend(loc='lower right')
ax0.set_ylabel('Accuracy')
ax0.set_ylim([0.5,1])
ax0.set_title('Training and Validation Accuracy')

ax1.plot(loss, label='Training Loss')
ax1.plot(val_loss, label='Validation Loss')
ax1.legend(loc='upper right')
ax1.set_ylabel('Cross Entropy')
ax1.set_title('Training and Validation Loss')
ax1.set_xlabel('epoch')
ax1.set_yscale("log")

Pourquoi, au début, les métriques de validation sont meilleures que les métriques de train?

* Raison principale: les layers `BatchNormalization` et `Dropout` affectent la précision pendant la phase Train.

* Raison secondaire: les métriques train sont des moyennes sur les différents `step` d'une époque. Tandis que les métriques de validation sont calculée a la fin de chaque époque; donc sur un modèle qui a été entrainé plus longtemps.

## Fine tuning

Pour améliorer les performances, on va maintenant dégeler les couches convolutives du haut (celles proche des couches denses ajoutées) et poursuivre l'entrainement. Cela va forcer les features-maps à s'adapter à notre problème de chats et chiens.


Attention: il ne faut faire cette opération de fine-tuning qu'après avoir déjà entrainé le nouveau classifier dense. Sinon: les gradients qui vont provenir du classifier aléatoire vont être trop grand, et vont détruire ce que les couches convolutives ont pré-apprises.




### Un-freeze the top layers of the model


In [47]:
base_model.trainable = True

MobilNet est très profond.

In [48]:
len(base_model.layers)

On va garder geler les 100 premières couches.

In [49]:
for layer in base_model.layers[:100]:
    layer.trainable =  False

### Compilons

On utilise maintenant un leraning rate plus petit

In [50]:
model.compile(loss=my_loss,
              optimizer = tf.optimizers.RMSprop(base_learning_rate/10),
              metrics=[my_accuracy])

In [51]:
len(model.trainable_variables)

### L'entrainement, c'est reparti

Ce second entrainement va améliorer l'accuracy de quelques pourcents.

In [52]:
fine_tune_epochs = 5
total_epochs =  initial_epochs + fine_tune_epochs

history_fine = model.fit(train_batches,
                         epochs=total_epochs,
                         initial_epoch =  history.epoch[-1],
                         validation_data=validation_batches)

On a un peu de sur-apprentissage car:
* Le jeu d'entrainement (chat-chien) est assez petits
* Le jeu d'entrainement (chat-chien) est assez similaire au jeu du pré-entrainement (image-net).


In [ ]:
acc += history_fine.history['my_accuracy']
val_acc += history_fine.history['val_my_accuracy']

loss += history_fine.history['loss']
val_loss += history_fine.history['val_loss']

In [ ]:
plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.ylim([0.8, 1])
plt.plot([initial_epochs-1,initial_epochs-1],
          plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.ylim([0, 1.0])
plt.plot([initial_epochs-1,initial_epochs-1],
         plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()